## Imports and config for aws sagemaker

In [1]:
#!conda update -n base -c defaults conda

In [2]:
#import boto3
#import sagemaker
#from sagemaker.amazon.common import numpy_to_record_serializer
#from sagemaker.predictor import csv_serializer, json_deserializer
#from sagemaker import get_execution_role
#from sagemaker.amazon.amazon_estimator import get_image_uri
#np.set_printoptions(precision=3, suppress=True)

In [3]:
#role = get_execution_role()
#bucket = 'better-reads-data'
#prefix = 'sagemaker/better-reads-lda'

#print(f'Training input/output will be stored in {bucket}/{prefix}')
#print(f'\nIAM Role: {role}')

In [4]:
#recordio_protobuf_serializer = numpy_to_record_serializer()
#fbuffer = recordio_protobuf_serializer(documents_training)

In [5]:
#%matplotlib inline
#import matplotlib.pyplot as plt

## Package used for language detection

In [6]:
!pip install langdetect

## Imports

In [7]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
import os
import string
import nltk
from langdetect import detect

## Package downloads for nltk

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cactuar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cactuar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Cactuar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cactuar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## NLP tools

In [9]:
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
words = set(nltk.corpus.words.words())

## Read in local csv

In [10]:
grdf = pd.read_csv('C:/Users/Cactuar/Downloads/15K_send_to_mh.csv', engine='python')

## Filtering books by english lanuage

In [11]:
def bad_entries(pandas_series):
    """
    finds non-english books and NaN desciption books
    and returns their row index in a list
    """
    non_eng = []
    for x,y in enumerate(pandas_series):
        try:
            z = detect(y)
        except:
            non_eng.append(x)
        if z != 'en':
            non_eng.append(x)
            
    return non_eng

In [12]:
books_to_remove = bad_entries(grdf['descrip'])

In [159]:
grdf = grdf.drop(books_to_remove, axis=0)

In [160]:
len(grdf)

12933

## Data prep for TF-IDF vectorizer

In [139]:
#grdf.drop_duplicates(subset='n', inplace=True)

In [161]:
grdf['descrip'] = grdf['descrip'].apply(lambda x: re.sub(r'[^a-zA-Z]'," ", x).strip())

In [162]:
grdf = grdf[grdf['descrip'].apply(lambda x: len(x) > 250)]

## Tokenization-Lemmatization

In [163]:
def token_lem(pandas_series):
    """
    returns a list of lemmatized tokens for vectorization
    from a pandas series
    """
    
    filtered_list = []
    
    #get tokens
    for val in pandas_series:
        tokens = word_tokenize(re.sub(r'[^a-zA-Z]'," ",val))
    
    # extra catching 
        filtered = []
        for word in tokens:
            word = word.lower()
            if word not in stop_words:
                if word in words:
                    if len(word) > 2:
                        if word not in filtered:
                            filtered.append(word)
    
    
    # lemmas
        lemmas = []        
        for word in filtered:
            word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
            word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
            word3 = wordnet_lemmatizer.lemmatize(word2, pos = "a")
            lemmas.append(word3.translate(str.maketrans('', '', string.punctuation)))
        filtered_list.append(lemmas)
    
    return filtered_list

In [167]:
filtered_list = []
for val in grdf['descrip']:
    tokens = word_tokenize(re.sub(r'[^a-zA-Z]'," ",val))
    #tokens = regex.sub('', tokens)
    filtered = []
    for word in tokens:
        word = word.lower()
        if word not in stop_words:
            if word in words:
                if len(word) > 2:
                    if word not in filtered:
                        filtered.append(word)
        
    lemmas = []        
    for word in filtered:
        word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
        word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
        word3 = wordnet_lemmatizer.lemmatize(word2, pos = "a")
        lemmas.append(word3.translate(str.maketrans('', '', string.punctuation)))
    filtered_list.append(lemmas)

In [164]:
lem_tokens = func(grdf['descrip'])

## Joining the words from tokenization

In [168]:
text = []
for w in filtered_list:
    text.append(" ".join(w))

## Instaniate vectorizer

In [174]:
tfidf = TfidfVectorizer(
    stop_words='english',
    lowercase=False,
    max_df=0.9,
    min_df=0.01,
    max_features=10000
)

## Vectorize and convert to dense matrix

In [175]:
sparse = tfidf.fit_transform(new_list)

In [176]:
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

In [177]:
dtm.shape

(49720, 498)

## Instantiate and fit the model

In [179]:
nn  = NearestNeighbors(n_neighbors=5, leaf_size=15, algorithm='auto', p=2, metric='euclidean')
nn.fit(dtm)

NearestNeighbors(algorithm='auto', leaf_size=15, metric='euclidean',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

### Sample text

In [180]:
samp = [
    """
    war civil history plantiff
    """
    
]

### Transform

In [181]:
new = tfidf.transform(samp)
new= new.todense()

## Predict

In [182]:
nn.kneighbors(new)

(array([[1., 1., 1., 1., 1.]]),
 array([[25712, 25708, 49717, 49718, 25709]], dtype=int64))

In [184]:
grdf['descrip'].iloc[25712]

IndexError: single positional indexer is out-of-bounds